## Implementation of PilotNet, with variation

Implement [PilotNet](https://arxiv.org/pdf/1604.07316.pdf) using Keras (with theano backend), while also trying different variations

In [19]:
import os
import pandas as pd
import numpy as np

from sklearn.utils import shuffle

In [20]:
import keras
from keras import backend as K
from keras.utils.data_utils import get_file
from keras.utils import np_utils
from keras.utils.np_utils import to_categorical
from keras.models import Sequential, Model
from keras.layers import Input, Embedding, Reshape, merge, LSTM, Bidirectional
from keras.layers import SpatialDropout1D, Concatenate  # Keras2

from keras.layers import TimeDistributed, Activation, SimpleRNN, GRU
from keras.layers.core import Flatten, Dense, Dropout, Lambda

from keras.regularizers import l2, l1

from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD, RMSprop, Adam

from keras.layers import deserialize
from keras.layers.merge import dot, add, concatenate
from keras.metrics import categorical_crossentropy, categorical_accuracy
from keras.layers.convolutional import *
from keras.preprocessing import image, sequence
from keras.preprocessing.text import Tokenizer

from keras.callbacks import ReduceLROnPlateau

### Check data format

check the csv output, and shuffle

In [2]:
%ls ../data/

9d0c3c2b.csv  img/


In [29]:
data_path = '../data/'
img_front_dir_path = '../data/img/front/'
model_path = '../model/'
cur_file = '9d0c3c2b'

In [5]:
df = pd.read_csv(os.path.join(data_path, cur_file+'.csv'))
df = shuffle(df)
df.head(3)

,id,img,wheel-axis,clutch,brake,gas,paddle-left,paddle-right,wheel-button-left-1,wheel-button-left-2,...,gear-1,gear-2,gear-3,gear-4,gear-5,gear-6,gear-R,front,side_left,side_right
1267,1267,9d0c3c2b_2017_07_27_14_57_29_96.jpg,24,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,9d0c3c2b_2017_07_27_14_57_29_96_front.jpg,9d0c3c2b_2017_07_27_14_57_29_96_left.jpg,9d0c3c2b_2017_07_27_14_57_29_96_right.jpg
335,335,9d0c3c2b_2017_07_27_14_55_56_62.jpg,-813,0,0,65535,0,0,0,0,...,0,0,0,0,0,0,0,9d0c3c2b_2017_07_27_14_55_56_62_front.jpg,9d0c3c2b_2017_07_27_14_55_56_62_left.jpg,9d0c3c2b_2017_07_27_14_55_56_62_right.jpg
1248,1248,9d0c3c2b_2017_07_27_14_57_28_05.jpg,-409,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,9d0c3c2b_2017_07_27_14_57_28_05_front.jpg,9d0c3c2b_2017_07_27_14_57_28_05_left.jpg,9d0c3c2b_2017_07_27_14_57_28_05_right.jpg


###  First, let's try to predict steering angle, using front image

define input and output, and load to memory. (should change to batch style code)

In [10]:
def img_to_arr(p):
    with image.load_img(p) as img:
        img = image.img_to_array(img)
    return img

In [11]:
X_train = np.array([img_to_arr(os.path.join(img_front_dir_path, p)) for p in df['front']])
y_train = df['wheel-axis'].values

input_shape = X_train.shape[1:]

In [31]:
X_train.shape

(1357, 3, 341, 562)

In [24]:
reduce_lr = ReduceLROnPlateau(patience=5, verbose=1, min_lr=1e-07)

In [25]:
def get_model_original():
    model = Sequential([
        Lambda(lambda x: x / 255.0 - 0.5, input_shape=input_shape, output_shape=input_shape),
        Conv2D(24, kernel_size=(5,5), strides=(2,2), activation='relu'),
        Conv2D(36, kernel_size=(5,5), strides=(2,2), activation='relu'),
        Conv2D(48, kernel_size=(5,5), strides=(2,2), activation='relu'),
        Conv2D(64, kernel_size=(3,3), strides=(1,1), activation='relu'),
        Conv2D(64, kernel_size=(3,3), strides=(1,1), activation='relu'),
        Flatten(),
        Dense(100, activation='relu'),
        Dense(50, activation='relu'),
        Dense(10, activation='relu'),
        Dense(1)
    ])
    
    model.compile(optimizer="adam", loss="mse") 
    model.summary()
    return model

In [26]:
model_original = get_model_original()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_3 (Lambda)            (None, 3, 341, 562)       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 24, 169, 279)      1824      
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 36, 83, 138)       21636     
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 48, 40, 67)        43248     
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 64, 38, 65)        27712     
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 64, 36, 63)        36928     
_________________________________________________________________
flatten_2 (Flatten)          (None, 145152)            0         
__________

In [27]:
model_original.fit(X_train, y_train, validation_split = 0.2, shuffle = True, epochs = 200, callbacks = [reduce_lr])

Train on 1085 samples, validate on 272 samples
Epoch 1/200
1085/1085 [==============================] - 4s - loss: 322856.8368 - val_loss: 178344.3093
Epoch 2/200
1085/1085 [==============================] - 4s - loss: 89878.9937 - val_loss: 53685.7809
Epoch 3/200
1085/1085 [==============================] - 4s - loss: 35448.5003 - val_loss: 33843.5757
Epoch 4/200
1085/1085 [==============================] - 4s - loss: 23121.1149 - val_loss: 16078.1961
Epoch 5/200
1085/1085 [==============================] - 4s - loss: 14546.1680 - val_loss: 10657.0036
Epoch 6/200
1085/1085 [==============================] - 4s - loss: 9217.5359 - val_loss: 8173.0681
Epoch 7/200
1085/1085 [==============================] - 4s - loss: 6850.3887 - val_loss: 7804.9262
Epoch 8/200
1085/1085 [==============================] - 4s - loss: 5530.7192 - val_loss: 4830.2567
Epoch 9/200
1085/1085 [==============================] - 4s - loss: 4611.8835 - val_loss: 5014.8635
Epoch 10/200
1085/1085 [=================

1085/1085 [==============================] - 4s - loss: 136.2487 - val_loss: 954.0954
Epoch 80/200
1085/1085 [==============================] - 6s - loss: 137.5724 - val_loss: 954.2149
Epoch 81/200
1085/1085 [==============================] - 6s - loss: 136.1583 - val_loss: 951.5834
Epoch 82/200
1085/1085 [==============================] - 6s - loss: 135.4220 - val_loss: 950.7741
Epoch 83/200
1085/1085 [==============================] - 6s - loss: 135.7186 - val_loss: 954.2925
Epoch 84/200
1056/1085 [============================>.] - ETA: 0s - loss: 134.3506
Epoch 00083: reducing learning rate to 1.00000006569e-06.
1085/1085 [==============================] - 6s - loss: 135.4220 - val_loss: 958.0565
Epoch 85/200
1085/1085 [==============================] - 6s - loss: 133.3969 - val_loss: 952.7108
Epoch 86/200
1085/1085 [==============================] - 6s - loss: 132.9185 - val_loss: 951.1785
Epoch 87/200
1085/1085 [==============================] - 6s - loss: 132.8839 - val_loss: 950

KeyboardInterrupt: 

Save the keras model

In [30]:
model_original.save(os.path.join(model_path, cur_file+'.h5'))

### Use batch normalization

Instead of normalizing the input ourselves, let's add a batch normalization layer as the first layer.

While we're at it, let's add bactch norm after each layer.

In [9]:
def get_model_batch_norm():
    model = Sequential([
        BatchNormalization(axis=1, input_shape=input_shape),
        Conv2D(24, kernel_size=(5,5), strides=(2,2), activation='relu'),
        BatchNormalization(axis=1),
        Conv2D(36, kernel_size=(5,5), strides=(2,2), activation='relu'),
        BatchNormalization(axis=1),
        Conv2D(48, kernel_size=(5,5), strides=(2,2), activation='relu'),
        BatchNormalization(axis=1),
        Conv2D(64, kernel_size=(3,3), strides=(1,1), activation='relu'),
        BatchNormalization(axis=1),
        Conv2D(64, kernel_size=(3,3), strides=(1,1), activation='relu'),
        BatchNormalization(axis=1),
        Flatten(),
        Dense(100, activation='relu'),
        BatchNormalization(),
        Dense(50, activation='relu'),
        BatchNormalization(),
        Dense(10, activation='relu'),
        BatchNormalization(),
        Dense(1)
    ])
    
    model.compile(optimizer="adam", loss="mse") 
    model.summary()
    return model

In [10]:
model_batch_norm = get_model_batch_norm()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_1 (Batch (None, 3, 160, 320)       12        
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 24, 78, 158)       1824      
_________________________________________________________________
batch_normalization_2 (Batch (None, 24, 78, 158)       96        
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 36, 37, 77)        21636     
_________________________________________________________________
batch_normalization_3 (Batch (None, 36, 37, 77)        144       
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 48, 17, 37)        43248     
_________________________________________________________________
batch_normalization_4 (Batch (None, 48, 17, 37)        192       
__________

In [12]:
model_batch_norm.fit(X_train, y_train, validation_split = 0.2, shuffle = True, epochs = 20, callbacks = [reduce_lr])

Train on 9742 samples, validate on 2436 samples
Epoch 1/20
9742/9742 [==============================] - 21s - loss: 0.0624 - val_loss: 0.0286
Epoch 2/20
9742/9742 [==============================] - 21s - loss: 0.0183 - val_loss: 0.0167
Epoch 3/20
9742/9742 [==============================] - 21s - loss: 0.0146 - val_loss: 0.0162
Epoch 4/20
9742/9742 [==============================] - 21s - loss: 0.0127 - val_loss: 0.0157
Epoch 5/20
9742/9742 [==============================] - 21s - loss: 0.0102 - val_loss: 0.0155
Epoch 6/20
9742/9742 [==============================] - 21s - loss: 0.0081 - val_loss: 0.0165
Epoch 7/20
9742/9742 [==============================] - 21s - loss: 0.0068 - val_loss: 0.0159
Epoch 8/20
9742/9742 [==============================] - 21s - loss: 0.0058 - val_loss: 0.0147
Epoch 9/20
9742/9742 [==============================] - 21s - loss: 0.0051 - val_loss: 0.0154
Epoch 10/20
9742/9742 [==============================] - 21s - loss: 0.0048 - val_loss: 0.0150
Epoch 11/20

In [13]:
model_batch_norm.fit(X_train, y_train, validation_split = 0.2, shuffle = True, epochs = 20, callbacks = [reduce_lr])

Train on 9742 samples, validate on 2436 samples
Epoch 1/20
9742/9742 [==============================] - 21s - loss: 0.0036 - val_loss: 0.0143
Epoch 2/20
9742/9742 [==============================] - 21s - loss: 0.0028 - val_loss: 0.0143
Epoch 3/20
9742/9742 [==============================] - 21s - loss: 0.0026 - val_loss: 0.0141
Epoch 4/20
9742/9742 [==============================] - 21s - loss: 0.0023 - val_loss: 0.0139
Epoch 5/20
9742/9742 [==============================] - 21s - loss: 0.0021 - val_loss: 0.0139
Epoch 6/20
9742/9742 [==============================] - 21s - loss: 0.0019 - val_loss: 0.0141
Epoch 7/20
9742/9742 [==============================] - 21s - loss: 0.0019 - val_loss: 0.0139
Epoch 8/20
9742/9742 [==============================] - 21s - loss: 0.0018 - val_loss: 0.0139
Epoch 9/20
9728/9742 [============================>.] - ETA: 0s - loss: 0.0017
Epoch 00008: reducing learning rate to 1.0000000475e-05.
9742/9742 [==============================] - 21s - loss: 0.0017 

As we can see, batch norm didn't help.